In [252]:
from pandas import read_csv 

from sklearn.feature_selection import RFE 

url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.csv" 

names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class'] 

dataframe = read_csv(url, names=names) 

array = dataframe.values 

X = array[:,0:8] 
Y = array[:,8] 

print(X)
print(Y)

[[  6.    148.     72.    ...  33.6     0.627  50.   ]
 [  1.     85.     66.    ...  26.6     0.351  31.   ]
 [  8.    183.     64.    ...  23.3     0.672  32.   ]
 ...
 [  5.    121.     72.    ...  26.2     0.245  30.   ]
 [  1.    126.     60.    ...  30.1     0.349  47.   ]
 [  1.     93.     70.    ...  30.4     0.315  23.   ]]
[1. 0. 1. 0. 1. 0. 1. 0. 1. 1. 0. 1. 0. 1. 1. 1. 1. 1. 0. 1. 0. 0. 1. 1.
 1. 1. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 1. 0. 1. 0. 0.
 1. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0. 0. 0. 1. 0.
 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0.
 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 1. 1. 1. 0. 0. 0.
 1. 0. 0. 0. 1. 1. 0. 0. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.
 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 1. 0. 0. 0. 1. 0. 0. 0. 0. 1. 1. 0. 0.
 0. 0. 1. 1. 0. 0. 0. 1. 0. 1. 0. 1. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 0. 0.
 1. 1. 0. 1. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 1. 1. 0. 1. 0. 0. 0. 1. 1. 1.
 1. 0. 

In [253]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,precision_score,recall_score
from sklearn.feature_selection import f_classif,RFE
from sklearn import tree

In [254]:
#split data

X_train,X_test,y_train,y_test = train_test_split(X,Y,test_size=0.2)

In [255]:
# use tree only

model = tree.DecisionTreeClassifier().fit(X_train,y_train)
y_hat = model.predict(X_test)
print('Accuracy_score',accuracy_score(y_test,y_hat))
print('Precision_score',precision_score(y_test,y_hat))
print('Recall_score',recall_score(y_test,y_hat))


Accuracy_score 0.7012987012987013
Precision_score 0.6
Recall_score 0.6206896551724138


In [256]:
# use anova 
import pandas as pd
feature_selected = f_classif(X_train,y_train)
print('f_socre :',feature_selected[0])
print('P-value :',feature_selected[1])

se1 = pd.Series(array[:,0])
se2 = pd.Series(array[:,1])
se3 = pd.Series(array[:,5])
se4 = pd.Series(array[:,7])
df = pd.DataFrame(se1,columns=['preg'])
df['plas'] = se2
df['mass'] = se3
df['age'] = se4
X = df
X_train,X_test,y_train,y_test = train_test_split(X,Y,test_size=0.2)

model = tree.DecisionTreeClassifier().fit(X_train,y_train)
y_hat = model.predict(X_test)
print('Accuracy_score',accuracy_score(y_test,y_hat))
print('Precision_score',precision_score(y_test,y_hat))
print('Recall_score',recall_score(y_test,y_hat))


f_socre : [ 28.92411723 151.86922689   2.52209281   1.00177994   6.42821477
  48.55341376  13.65019429  36.2640213 ]
P-value : [1.07254057e-07 2.50307071e-31 1.12777975e-01 3.17275765e-01
 1.14799181e-02 8.33366819e-12 2.39942099e-04 2.97276324e-09]
Accuracy_score 0.6298701298701299
Precision_score 0.453125
Recall_score 0.5686274509803921


In [257]:
# use RFE
import numpy as np
from sklearn.model_selection import KFold
kf = KFold(n_splits=8)
kf.get_n_splits(dataframe)
array = dataframe.values 
X = array[:,0:8] 
Y = array[:,8] 
n_feature = [1,2,3,4,5,6,7,8]
n_score =[]
i=0
model = tree.DecisionTreeClassifier()
for train_index, test_index in kf.split(X):
    X_train, X_test,y_train, y_test  = X[train_index],X[test_index],Y[train_index],Y[test_index]
    selector = RFE(model,n_features_to_select=n_feature[i],step=1)
    selector.fit(X_train,y_train)
    print("round {} =".format(i),selector.support_)
    print("round {} =".format(i),selector.ranking_)
    y_hat=selector.predict(X_test)
    n_score.append((accuracy_score(y_test,y_hat)))
    i+=1

print("Accuracy_score ของแต่ละ feature\n",n_score)
map_feature = n_feature[np.argmax(n_score)]
print("the best feature = {}".format(map_feature))
# print(map_feature)

X_train,X_test,y_train,y_test = train_test_split(X,Y,test_size=0.2)
model.fit(X_train,y_train)
y_hat=model.predict(X_test)


print('Accuracy_score',accuracy_score(y_test,y_hat))
print('Precision_score',precision_score(y_test,y_hat))
print('Recall_score',recall_score(y_test,y_hat))

round 0 = [False  True False False False False False False]
round 0 = [6 1 4 8 7 3 2 5]
round 1 = [False  True False False False  True False False]
round 1 = [6 1 4 7 5 1 2 3]
round 2 = [False  True False False False  True False  True]
round 2 = [5 1 3 4 6 1 2 1]
round 3 = [False  True  True False False  True  True False]
round 3 = [4 1 1 5 3 1 1 2]
round 4 = [False  True  True False False  True  True  True]
round 4 = [3 1 1 4 2 1 1 1]
round 5 = [False  True  True False  True  True  True  True]
round 5 = [3 1 1 2 1 1 1 1]
round 6 = [ True  True  True False  True  True  True  True]
round 6 = [1 1 1 2 1 1 1 1]
round 7 = [ True  True  True  True  True  True  True  True]
round 7 = [1 1 1 1 1 1 1 1]
Accuracy_score ของแต่ละ feature
 [0.65625, 0.65625, 0.53125, 0.6770833333333334, 0.6458333333333334, 0.8020833333333334, 0.7395833333333334, 0.75]
the best feature = 6
Accuracy_score 0.7792207792207793
Precision_score 0.6166666666666667
Recall_score 0.7708333333333334
